In [1]:
!pip uninstall numpy
!pip install -U numpy
!pip install boto3

Found existing installation: numpy 1.21.5
Uninstalling numpy-1.21.5:
  Would remove:
    /usr/bin/f2py
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.7
    /usr/local/lib/python3.7/dist-packages/numpy-1.21.5.dist-info/*
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libgfortran-2e0d59d6.so.5.0.0
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libopenblasp-r0-2d23e62b.3.17.so
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libquadmath-2d0c479f.so.0.0.0
    /usr/local/lib/python3.7/dist-packages/numpy/*
Proceed (y/n)? y
  Successfully uninstalled numpy-1.21.5
     |████████████████████████████████| 15.7 MB 4.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0

     |████████████████████████████████| 132 kB 4.3 MB/s 
     |████████████████████████████████| 79 kB 9.4 MB/s 
     |████████████████████████████████| 8.6 MB 36.6 MB/s 
     |████████████████████████████████| 138 kB 44.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/Bert-Chinese-Text-Classification-Pytorch-master')

sys.argv=['']
del sys

Mounted at /content/drive


In [3]:
from sklearn.model_selection import train_test_split
f=open('/content/drive/MyDrive/Bert-Chinese-Text-Classification-Pytorch-master/THUCNews/data/all_data_with_label.txt','r',encoding='utf-8')
data=f.readlines()
f.close()

In [4]:
train_set,test_set=train_test_split(data,test_size=0.4)
dev_set,test_set=train_test_split(test_set,test_size=0.4)

In [5]:
f=open('/content/drive/MyDrive/Bert-Chinese-Text-Classification-Pytorch-master/THUCNews/data/train.txt','w',encoding='utf-8')
for i in train_set:
  f.write(i)
f.close()

f=open('/content/drive/MyDrive/Bert-Chinese-Text-Classification-Pytorch-master/THUCNews/data/dev.txt','w',encoding='utf-8')
for i in dev_set:
  f.write(i)
f.close()

f=open('/content/drive/MyDrive/Bert-Chinese-Text-Classification-Pytorch-master/THUCNews/data/test.txt','w',encoding='utf-8')
for i in test_set:
  f.write(i)
f.close()

In [11]:
# coding: UTF-8
import torch
import torch.nn as nn
# from pytorch_pretrained_bert import BertModel, BertTokenizer
from pytorch_pretrained import BertModel, BertTokenizer


class Config(object):

    """配置参数"""
    def __init__(self, dataset):
        self.model_name = 'bert'
        self.train_path = dataset + '/data/train.txt'                                # 训练集
        self.dev_path = dataset + '/data/dev.txt'                                    # 验证集
        self.test_path = dataset + '/data/test.txt'                                  # 测试集
        self.class_list = [x.strip() for x in open(
            dataset + '/data/class_my.txt').readlines()]                                # 类别名单
        self.save_path = dataset + '/saved_dict/' + self.model_name + '_without_my.ckpt'        # 模型训练结果
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   # 设备

        self.require_improvement = 1000                                 # 若超过1000batch效果还没提升，则提前结束训练
        self.num_classes = len(self.class_list)                         # 类别数
        #self.num_epochs = 3                                             # epoch数
        self.num_epochs = 25
        #self.batch_size = 128
        self.batch_size = 8                                           # mini-batch大小
        self.pad_size = 32                                              # 每句话处理成的长度(短填长切)
        self.learning_rate = 5e-5                                       # 学习率
        self.bert_path = 'bert-base-chinese'
        self.tokenizer = BertTokenizer.from_pretrained(self.bert_path)
        self.hidden_size = 768


class Model(nn.Module):

    def __init__(self, config):
        super(Model, self).__init__()
        self.bert = BertModel.from_pretrained(config.bert_path)
        for param in self.bert.parameters():
            param.requires_grad = True
        self.fc = nn.Linear(config.hidden_size, config.num_classes)

    def forward(self, x):
        context = x[0]  # 输入的句子
        mask = x[2]  # 对padding部分进行mask，和句子一个size，padding部分用0表示，如：[1, 1, 1, 1, 0, 0]
        _, pooled = self.bert(context, attention_mask=mask, output_all_encoded_layers=False)
        out = self.fc(pooled)
        return out


In [12]:
# coding: UTF-8
import time
import torch
import numpy as np
from train_eval import train, init_network
import argparse
from utils import build_dataset, build_iterator, get_time_dif

parser = argparse.ArgumentParser(description='Chinese Text Classification')
#parser.add_argument('--model', type=str, required=True, help='choose a model: Bert, ERNIE')
parser.add_argument('--model',default='bert',help='Bert')
args = parser.parse_args()


if __name__ == '__main__':
    dataset = '/content/drive/MyDrive/Bert-Chinese-Text-Classification-Pytorch-master/THUCNews'  # 数据集

    model_name = args.model  # bert
    config = Config(dataset)
    np.random.seed(1)
    torch.manual_seed(1)
    torch.cuda.manual_seed_all(1)
    torch.backends.cudnn.deterministic = True  # 保证每次结果一样

    start_time = time.time()
    print("Loading data...")
    train_data, dev_data, test_data = build_dataset(config)
    train_iter = build_iterator(train_data, config)
    dev_iter = build_iterator(dev_data, config)
    test_iter = build_iterator(test_data, config)
    time_dif = get_time_dif(start_time)
    print("Time usage:", time_dif)

    # train
    model = Model(config).to(config.device)
    dev_log,train_log = train(config, model, train_iter, dev_iter, test_iter)

Loading data...


400it [00:00, 12840.16it/s]
160it [00:00, 9759.02it/s]
107it [00:00, 10716.87it/s]

Time usage: 0:00:00


Epoch [1/25]
Iter:      0,  Train Loss:   1.5,  Train Acc: 25.00%,  Val Loss:   1.5,  Val Acc: 15.00%,  Time: 0:00:02 *
Iter:     10,  Train Loss:   1.1,  Train Acc: 62.50%,  Val Loss:  0.87,  Val Acc: 66.88%,  Time: 0:00:05 *
Iter:     20,  Train Loss:   1.2,  Train Acc: 50.00%,  Val Loss:  0.57,  Val Acc: 78.75%,  Time: 0:00:07 *
Iter:     30,  Train Loss:  0.39,  Train Acc: 87.50%,  Val Loss:  0.23,  Val Acc: 93.12%,  Time: 0:00:10 *
Iter:     40,  Train Loss:  0.52,  Train Acc: 75.00%,  Val Loss:  0.23,  Val Acc: 91.25%,  Time: 0:00:13 *
Epoch [2/25]
Iter:     50,  Train Loss:  0.26,  Train Acc: 75.00%,  Val Loss:  0.24,  Val Acc: 88.75%,  Time: 0:00:14 
Iter:     60,  Train Loss:  0.92,  Train Acc: 87.50%,  Val Loss:  0.23,  Val Acc: 91.25%,  Time: 0:00:15 
Iter:     70,  Train Loss:  0.63,  Train Acc: 75.00%,  Val Loss:  0.17,  Val Acc: 93.75%,  Time: 0:00:18 *
Iter:     80,  Train Loss:  0.39,  Train Acc: 87.50%,  Val Loss:  0.14,  Val Acc: 95.62%,  Time: 0:00:21 *
Iter:     90,

In [13]:
torch.save(model,'/content/drive/MyDrive/Bert-Chinese-Text-Classification-Pytorch-master/THUCNews/saved_dict/model')

In [14]:
import json

In [15]:
train_=[]
for i in train_log:
  train_.append(i.item())

dev_=[]
for i in dev_log:
  dev_.append(i.item())

In [16]:
filename='/content/drive/MyDrive/Bert-Chinese-Text-Classification-Pytorch-master/THUCNews/saved_dict/train_log.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(train_,file_obj,ensure_ascii=False,indent = 4)

filename='/content/drive/MyDrive/Bert-Chinese-Text-Classification-Pytorch-master/THUCNews/saved_dict/dev_log.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(dev_,file_obj,ensure_ascii=False,indent = 4)